In [3]:
import numpy as np
import re
import random
from collections import defaultdict
import csv

In [8]:
def mapdict(dict,sentence):
    for i in sentence:
        try:
            dict[i]+=1
        except:
            dict[i]=1
    return 
def getcount(word,content):
   
    dict={}
    size=len(content)
    for i in range(size):
            mapdict(dict,content[i])        
    return dict

def addunkown(dictionary,vocab):
    count=0
    keylist=list(dictionary.keys())
    for k in keylist:
        if k in vocab:
            continue
        else:
            count+=dictionary[k]
            del dictionary[k]  
    dictionary['UNK']=count

    return dictionary

def preprocess(content):
     elements=content.strip('\n').split("\t")
     return elements


def maptoken(dictionary,tokens):
    for i in range(len(tokens)):
        try:
            dictionary[tokens[i]]
        except:
            tokens[i]="UNK"
    return tokens

#smoothing of generation prob
def addksmoothing1(dict,tag_dict,bio_list,word_list,train_word,train_bio,k):
    V=len(word_list)
    for word in word_list:
        for t in bio_list:
            dict[(word,t)]=k/(tag_dict[t]+k*V)
    #map probabilty 
    for i in range(len(train_word)):
        for j in range(len(train_word[i])):
            word=train_word[i][j]
            t=train_bio[i][j]
            dict[(word,t)]+=1/(tag_dict[t]+k*V)
    
    return dict

#smoothing of transiton prob
def addksmoothing2(dict,bio_list,train_bio,k):
#a=Counter(list(zip(*train_bio))[0])
    b=bio_list[:]
    bigram_dict={}
    n_bio=len(b)
    b.append("#")
    for t1 in b:
        for t2 in b[:-1]:
            dict[(t1,t2)]=k
        #map probabilty 
    for i in range(len(train_bio)):
        sequence=["#"]+train_bio[i]
        bigram = [tuple(sequence[i:i+2]) for i in range(len(sequence)-(1))]
        for key in bigram:

            dict[key]+=1
            try:
                bigram_dict[key[0]]+=1
            except:
                bigram_dict[key[0]]=1
    for t1 in b:
        for t2 in b[:-1]:
            dict[(t1,t2)]= dict[(t1,t2)]/(k*n_bio+bigram_dict[t1]) 
    return dict

#hmm veterbi 
def viterbi_hmm(line,bio_list,word_tag,tag_tag):
  
    N=len(bio_list) #state-graph of len 
    T=len(line) #observation len T
    viterbi=np.zeros((N,T))
    bp = np.zeros((N,T),dtype=int)
    for i in range(N):
        viterbi[i][0]=word_tag[(line[0],bio_list[i])]*tag_tag[("#",bio_list[i])]
        bp[i][0]=0
    for t in range(1,T):
        for s in range(N):
            prob_to_state=np.zeros(N)
            for j in range (N):
                prob_to_state[j]=viterbi[j][t-1]*tag_tag[(bio_list[j],bio_list[s])]*word_tag[(line[t],bio_list[s])]
            viterbi[s][t]=np.max(prob_to_state)
            bp[s][t]=np.argmax(prob_to_state)
    path = []
    tag_index = int(viterbi.argmax(axis=0)[T-1])

    for i in reversed(range(T)):     
            path.append(bio_list[tag_index])
            tag_index = bp[tag_index][i]

    path = path[::-1]
    return path

#evalutation to compute precision ,recall and f1 score given by two inputs: predicted seq and correct seq
def evaluate_model_span(predicted_seq, correct_seq):
    T = len(predicted_seq)
    
    num_spans_pred = ['B-' in x for x in predicted_seq].count(True)
    num_spans_ans = ['B-' in x for x in correct_seq].count(True)
    
    correct_pred=0
    
    for i in range(len(correct_seq)):
        
        if 'B-' in correct_seq[i]:
            if correct_seq[i]==predicted_seq[i]:
                flag=1;
                tag_type = correct_seq[i][2:]
                j=i+1
                while( j<T and (correct_seq[j] == 'I-'+tag_type or predicted_seq[j] == 'I-'+tag_type)):

                    if (correct_seq[j]!=predicted_seq[j]):
                        flag=0;
                    
                    j=j+1             
                
                if(flag==1):
                    correct_pred+=1;
                               
                i=j-1
     
    precision = correct_pred/num_spans_pred;
    recall = correct_pred/num_spans_ans;
    
    f_score = (2*precision*recall)/(precision+recall)
                
    return precision, recall, f_score   

#train test split 
def train_test_split(content,coef):
    from sklearn.model_selection import train_test_split
    n=len(content)
    word=content[::3]
    pos=content[1::3]
    bio=content[2::3]
    word=[preprocess(i)for i in word]
    pos=[preprocess(i)for i in pos]
    bio=[preprocess(i)for i in bio]
    word_train, word_test, bio_train, bio_test = train_test_split(word, bio, test_size=1-coef, random_state=42)
    word_train, word_test, pos_train, pos_test = train_test_split(word, pos, test_size=1-coef, random_state=42)
    return  word_train,word_test,bio_train,bio_test

#generate NER index 
def gen_tag_indexes(pred):
    result = defaultdict(list)
    T = len(pred)
    for i in range(len(pred)):

        if 'B-' in pred[i]:
            flag=1;
            tag_type = pred[i][2:]
            j=i+1
            while j<T and (pred[j] == 'I-'+tag_type) :
                j=j+1             

            j=j-1 #To get to last token


            result[tag_type].append(str(i)+'-'+str(j))
    return result


#memm one feature
def viterbi_memm(line,bio_list,dict,count_dict,k):
    #print(line)
    N=len(bio_list) #state-graph of len 
    T=len(line) #observation len T
    viterbi=np.zeros((N,T))
    bp = np.zeros((N,T),dtype=int)
    for i in range(N):
        viterbi[i][0]=getaddkprob((line[0],"#"),bio_list[i],dict,count_dict,bio_list,k)
        bp[i][0]=0
    for t in range(1,T):
        for s in range(N):
            prob_to_state=np.zeros(N)
            for j in range (N):
                
                prob_to_state[j]=viterbi[j][t-1]*getaddkprob((line[t],bio_list[j]),bio_list[s],dict,count_dict,bio_list,k)
            viterbi[s][t]=np.max(prob_to_state)
            bp[s][t]=np.argmax(prob_to_state)
    path = []
    tag_index = int(viterbi.argmax(axis=0)[T-1])

    for i in reversed(range(T)):     
            path.append(bio_list[tag_index])
            tag_index = bp[tag_index][i]

    path = path[::-1]
    return path
#write result
def write_result(filename,preds):
    with open(filename, 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['Type','Prediction'])
        
        filewriter.writerow(['ORG',' '.join(preds['ORG'])])
        filewriter.writerow(['MISC',' '.join(preds['MISC'])])
        filewriter.writerow(['PER',' '.join(preds['PER'])])
        filewriter.writerow(['LOC',' '.join(preds['LOC'])])

In [6]:
#open the file
filename="train.txt"
f=open(filename,'r',encoding="utf8")
content=f.readlines()

#train test split (0.8:0.2)
word_train,word_test,bio_train,bio_test=train_test_split(content,0.8)
word={}
biotag={}
word=getcount(word,word_train)
biotag=getcount(biotag,bio_train)
word_sorted=sorted(word.items(),key=lambda x: (x[1]))
word_sorted=dict(word_sorted)
vocab1=[i for i,x in word_sorted.items() if x>0]


import time
start_time = time.time()
ff=-1
i=0

#random serach for best k1 and k2
iter=1
while(i<=iter):
            #set search range
            k1=random.uniform(0,1)
            k2=random.uniform(5, 7)    
        
            #create vocab where 100 words in the training set with count 1 will be assigned as unkown 
            k=100   
            vocab2=np.array([i for i,x in word_sorted.items() if x==1])
            t=random.sample(range(len(vocab2)), k)
            vocab3=vocab2[t]
            d=set(vocab1)-set(vocab3)
            vocab=list(d)
            print("Vocab:")
            print(len(vocab))
            
            
            #update the dictionary add unkown key and map unknown key in the training set
            word_sorted_un={}
            word_sorted_un=addunkown(word_sorted,vocab)
            word_train=[maptoken(word_sorted_un,i) for i in word_train]
            #apply smoothing
            bio_list=list(biotag.keys())
            word_list=list(word_sorted_un.keys())
            
            
            #map the unk key in the test set
            #process the dev set
            word_test=[maptoken(word_sorted_un,i) for i in word_test]
            
            #create generation prob dict and smooth with add-k smoothing. parameter :k1
            word_tag={}
            word_tag=addksmoothing1(word_tag,biotag,bio_list,word_list,word_train,bio_train,k1)
            
            #create transition prob dict  smooth with add-k smoothing. parameter :k2
            tag_tag={}
            tag_tag=addksmoothing2(tag_tag,bio_list,bio_train,k2)

            #generate predicted sequence using HMM
            predicted_seq=[]
            for line in word_test:
                predicted_seq+=viterbi_hmm(line,bio_list,word_tag,tag_tag)
            predicted_seq
            
            #generate correct sequence
            correct_seq=[]
            for line in bio_test:
                correct_seq+=line
            p,r,f=evaluate_model_span(predicted_seq, correct_seq)
            
            
            #find the best score
            if(f>ff):
                f_p=p
                f_r=r
                f_k=k
                f_k1=k1
                f_k2=k2
                ff=f
            i+=1
            print("iter:"+str(i)+": "+str(p)+"  "+str(r)+" "+str(f)+" with k parameters:  k :"+str(k)+" k1:"+str(k1)+" k2:"+str(k2)) 
print("Best performance: "+str(f_p)+"  "+str(f_r)+" "+str(ff)+" with k parameters:  k :"+str(f_k)+" k1:"+str(f_k1)+" k2:"+str(f_k2))

#report running time of number of iteraions
print("Running time--- %s seconds ---" % (time.time() - start_time))

Vocab:
21537
iter:1: 0.8436603557085485  0.6307098434484237 0.7218063566081727 with k parameters:  k :100 k1:0.8826556612078669 k2:5.390206333243154
Vocab:
21537
iter:2: 0.8424214417744916  0.7819000643362642 0.8110332554776999 with k parameters:  k :100 k1:0.0514534033509737 k2:5.005990672459048
Best performance: 0.8424214417744916  0.7819000643362642 0.8110332554776999 with k parameters:  k :100 k1:0.0514534033509737 k2:5.005990672459048
Running time--- 25.52758765220642 seconds ---


In [7]:
#combine the training and validation set and generate prediction 
word_train,word_test,bio_train,bio_test=train_test_split(content,1)
word={}
biotag={}
word=getcount(word,word_train)
biotag=getcount(biotag,bio_train)
word_sorted=sorted(word.items(),key=lambda x: (x[1]))
word_sorted=dict(word_sorted)
vocab1=[i for i,x in word_sorted.items() if x>0]


#set parameters we gained from training 
k=100  #unknown word
k1=0.03 # smoothing parameter for generation prob
k2=6.1  #smoothing parameter for transition prob


vocab2=np.array([i for i,x in word_sorted.items() if x==1])
t=random.sample(range(len(vocab2)), k)
vocab3=vocab2[t]
d=set(vocab1)-set(vocab3)
vocab=list(d)

print("Vocab")
print(len(vocab))
#update the dictionary
word_sorted_un={}
word_sorted_un=addunkown(word_sorted,vocab)
word_train=[maptoken(word_sorted_un,i) for i in word_train]
word_test=[maptoken(word_sorted_un,i) for i in word_test]
#apply smoothing
bio_list=list(biotag.keys())
word_list=list(word_sorted_un.keys())

#update the dictionary add unkown key and map unknown key in the training set
word_sorted_un={}
word_sorted_un=addunkown(word_sorted,vocab)
word_train=[maptoken(word_sorted_un,i) for i in word_train]
#apply smoothing
bio_list=list(biotag.keys())
word_list=list(word_sorted_un.keys())


#create generation prob dict and smooth with add-k smoothing. parameter :k1
word_tag={}
word_tag=addksmoothing1(word_tag,biotag,bio_list,word_list,word_train,bio_train,k1)

#create transition prob dict  smooth with add-k smoothing. parameter :k2
tag_tag={}
tag_tag=addksmoothing2(tag_tag,bio_list,bio_train,k2)

#generate predicted sequence using HMM
predicted_seq=[]
for line in word_test:
    predicted_seq+=viterbi_hmm(line,bio_list,word_tag,tag_tag)


#test file processing
filename="test.txt"
f=open(filename,'r',encoding="utf8")
content=f.readlines()
n=len(content)
word=content[::3]
pos=content[1::3]
index=content[2::3]
word_test=[preprocess(i)for i in word]
pos_test=[preprocess(i)for i in pos]
index_test=[preprocess(i)for i in index]
word_test=[maptoken(word_sorted_un,i) for i in word_test]

#generatee prediction
preds=gen_tag_indexes(predicted_seq)

#write results to file
write_result('hmm.csv', preds)
print("DONE")

Vocab
24239
DONE
